#Instalacion de las librerías necesarias:

gradio: UI web para el chat.

groq: API cliente del modelo LLM de Groq.

pinecone-client: para vectores e índices.

sentence-transformers: para generar embeddings.

langchain-text-splitters: para dividir texto.

pypdf: para leer PDFs.

In [1]:
!pip -q install gradio groq pinecone-client sentence-transformers langchain-text-splitters pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 10.4 MB/s eta 0:00:00


In [2]:
!pip install groq

In [3]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [1]:
import os
import re
import glob
from pathlib import Path
from typing import List, Dict, Any, Optional
from datetime import datetime

import gradio as gr
from groq import Groq
from google.colab import userdata

from pinecone import (
    Pinecone,
    ServerlessSpec,
    CloudProvider,
    AwsRegion,
    VectorType
)

from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pypdf import PdfReader


# Configuruacion y testeos

Obtiene las API keys desde los "secrets" de Colab. Si no existen, lanza un error.

In [ ]:
# =========================
# Secrets (Colab) - con fallback a input
# =========================
from google.colab import userdata
import os

def get_secret_or_input(key_name: str, prompt: str) -> str:
    """Intenta obtener un secreto de Colab.
    Si no existe, lo pide por consola."""
    try:
        value = userdata.get(key_name)
        if value:
            return value.strip()
    except Exception:
        # SecretNotFoundError u otro
        pass
    # Si no está en userdata, lo pide manualmente
    return input(f"👉 Ingresá tu {prompt}: ").strip()

GROQ_API_KEY = get_secret_or_input("GROQ_API_KEY", "GROQ_API_KEY")
PINECONE_API_KEY = get_secret_or_input("PINECONE_API_KEY", "PINECONE_API_KEY")

# Guardar en variables de entorno
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

print("✅ Claves configuradas correctamente")


In [4]:
# =========================
# Test GROQ
# =========================
import os
from groq import Groq

assert os.environ.get("GROQ_API_KEY"), "❌ Falta GROQ_API_KEY en os.environ"

try:
    client = Groq(api_key=os.environ["GROQ_API_KEY"])
    chat = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": "Sos un asistente muy conciso."},
            {"role": "user", "content": "Decime 'OK GROQ' si me leés bien."}
        ],
        max_tokens=8,
        temperature=0.0,
    )
    print("✅ GROQ OK:", chat.choices[0].message.content.strip())
except Exception as e:
    print("❌ Error GROQ:", type(e).__name__, str(e))
    raise

✅ GROQ OK: OK GROQ


In [5]:
from pinecone import Pinecone, ServerlessSpec
import os, time, uuid, numpy as np

In [6]:
def pinecone_reset_all(api_key=None, *, realmente_borrar=False, wait=True, timeout=120, poll_every=2.0):
    """
    Borra TODOS los índices de Pinecone (irreversible).
    - Si `realmente_borrar=False` (por defecto), solo muestra qué borraría (dry-run).
    - Si `wait=True`, espera a que desaparezcan del listado (hasta `timeout` segundos).

    Uso:
      # ver qué hay (dry-run)
      pinecone_reset_all()

      # borrar de verdad
      pinecone_reset_all(realmente_borrar=True)
    """
    import os, time
    from pinecone import Pinecone

    # Obtener API key (env o Colab)
    api_key = api_key or os.getenv("PINECONE_API_KEY")
    if not api_key:
        try:
            from google.colab import userdata
            api_key = userdata.get("PINECONE_API_KEY")
        except Exception:
            pass
    if not api_key:
        raise ValueError("Falta PINECONE_API_KEY (en entorno o en Colab > Secrets).")

    pc = Pinecone(api_key=api_key)

    items = pc.list_indexes()

    # Compatibilidad con distintas formas de listado
    def _name(x):
        if isinstance(x, str): return x
        n = getattr(x, "name", None)
        if isinstance(n, str): return n
        if isinstance(x, dict): return x.get("name")
        return None

    names = [n for n in map(_name, items) if n]

    if not realmente_borrar:
        return {"accion": "dry-run", "encontrados": names}

    for n in names:
        pc.delete_index(n)

    if not wait:
        return {"accion": "deleted", "borrados": names}

    # Polling hasta que no aparezcan
    t0 = time.time()
    remaining = names[:]
    while time.time() - t0 < timeout:
        now = [n for n in map(_name, pc.list_indexes()) if n]
        remaining = [n for n in remaining if n in now]
        if not remaining:
            break
        time.sleep(poll_every)

    return {"accion": "deleted", "borrados": names, "restantes": remaining}


In [7]:
# 1) Ver qué índices hay (no borra)
pinecone_reset_all()


{'accion': 'dry-run', 'encontrados': ['cv--cv1', 'cv--cv2']}

In [8]:
# 2) Borrar todo y esperar confirmación
pinecone_reset_all(realmente_borrar=True)

{'accion': 'deleted', 'borrados': ['cv--cv1', 'cv--cv2'], 'restantes': []}

In [9]:
# 1) Ver qué índices hay (no borra)
pinecone_reset_all()

{'accion': 'dry-run', 'encontrados': []}

#Definicion:

Directorio de documentos (/content/cv)

Modelo de embeddings (MiniLM)

Tamaño y solapamiento de chunks

Región y proveedor para Pinecone

Parámetros default para generación (top_p, temperature, max_tokens)

In [10]:
# Carpeta destino
!mkdir -p /content/cv

In [11]:
# Lista de URLs (una por línea)
urls = [
    "https://raw.githubusercontent.com/ctalamilla/UBA-NLP2-TP3/refs/heads/main/cv2.txt",
    "https://raw.githubusercontent.com/ctalamilla/UBA-NLP2-TP3/refs/heads/main/cv1.txt"
    ]

# Descargar todos en la carpeta /content/cv
for u in urls:
    !wget -nc -P /content/cv "{u}"

--2025-08-24 13:50:34--  https://raw.githubusercontent.com/ctalamilla/UBA-NLP2-TP3/refs/heads/main/cv2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1389 (1.4K) [text/plain]
Saving to: ‘/content/cv/cv2.txt’

cv2.txt             100%[===================>]   1.36K  --.-KB/s    in 0s      

2025-08-24 13:50:34 (17.3 MB/s) - ‘/content/cv/cv2.txt’ saved [1389/1389]

--2025-08-24 13:50:34--  https://raw.githubusercontent.com/ctalamilla/UBA-NLP2-TP3/refs/heads/main/cv1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1405 (1.4K)

In [49]:
DOCS_DIR = "/content/cv"                   # (.pdf / .txt)
INDEX_PREFIX = "cv--"                       # Prefijo para índices por CV
EMB_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
CHUNK_SIZE = 200
CHUNK_OVERLAP = 25
EMB_DIM = 384                              # all-MiniLM-L6-v2 -> 384
PINECONE_CLOUD = CloudProvider.AWS
PINECONE_REGION = AwsRegion.US_EAST_1

In [50]:
# Parámetros de generación por defecto (Se ajustan desde la interfase)
TOP_K_DEFAULT = 4
TEMPERATURE_DEFAULT = 0.7
TOP_P_DEFAULT = 0.9
MAX_TOKENS_DEFAULT = 512

SYSTEM_PROMPT = (
    "Eres un asistente que responde usando EXCLUSIVAMENTE el contexto proporcionado. "
    "Si el contexto no contiene la respuesta, dilo con claridad. Sé breve, claro y útil."
)

ROUTER_SYSTEM = (
    "Eres un enrutador. Tu tarea es elegir UN ÚNICO índice de la lista que mejor "
    "corresponda a la consulta del usuario. Responde SOLO con el nombre EXACTO del índice y nada más. "
    "Si ninguno aplica, responde 'NONE'."
)


In [51]:
# =========================
# Groq client
# =========================
client = Groq(api_key=GROQ_API_KEY)

# =========================
# Pinecone client
# =========================
pc = Pinecone(api_key=PINECONE_API_KEY)

In [52]:
import re
import unicodedata
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

#Embeddings y Chunking

Divide texto largo en chunks superpuestos.

Calcula los embeddings de cada chunk (o uno solo).


In [55]:
# Embeddings
emb_model = SentenceTransformer(EMB_MODEL_NAME)

In [56]:
# Chunking
def chunk_text(text: str) -> List[str]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        separators=[" ", ""]
    )
    return splitter.split_text(text)

def embed_texts(texts: List[str]) -> List[List[float]]:
    return [vec.tolist() for vec in emb_model.encode(texts)]

def embed_text(text: str) -> List[float]:
    return emb_model.encode([text])[0].tolist()

#Limpieza del texto

Elimina tildes y diéresis (usando unicodedata.normalize).

Convierte la ñ a n.

Elimina caracteres no ASCII y nulos.

Reduce espacios múltiples a uno solo.

In [54]:
# Limpieza del texto

def clean_text(text: str) -> str:
    # Reemplaza caracteres acentuados por sus equivalentes no acentuados
    text = unicodedata.normalize('NFKD', text)
    text = "".join([c for c in text if not unicodedata.combining(c)])
    # Reemplazar ñ y Ñ por n y N
    text = text.replace("ñ", "n").replace("Ñ", "N")
    # Eliminar caracteres no ASCII
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    # Eliminar nulos
    text = text.replace("\x00", "")
    # Colapsar múltiples espacios
    text = re.sub(r"\s+", " ", text)
    return text.strip()


#Lectura de archivos

Lee archivos .txt como latin-1.

Lee PDFs usando PdfReader.

Usa clean_text() en ambos casos.

read_file() elige el método según la extensión.

In [57]:
# Lectura de archivos

def read_txt(path: str) -> str:
    with open(path, "r", encoding="latin-1", errors="ignore") as f:
        raw = f.read()
        return clean_text(raw)

def read_pdf(path: str) -> str:
    text_parts = []
    try:
        reader = PdfReader(path)
        for page in reader.pages:
            text_parts.append(page.extract_text() or "")
    except Exception:
        pass
    return clean_text("\n".join(text_parts))

def read_file(path: Path) -> str:
    ext = path.suffix.lower()
    if ext == ".txt":
        return read_txt(str(path))
    elif ext == ".pdf":
        return read_pdf(str(path))
    return ""


In [58]:
clean_text(read_txt('/content/cv/cv1.txt'))

'# CV Corporativo - Ricardo Lopez ## Datos Personales - Nombre: Ricardo Lopez - Edad: 47 anios - Correo: ricardo.lopez.corporativo@example.com - Ubicacion: Buenos Aires, Argentina - LinkedIn: linkedin.com/in/ricardolopez-corporativo ## Resumen Profesional - Ejecutivo con experiencia en liderazgo empresarial y gestion de proyectos tecnologicos. - Especialista en transformacion digital y gestion de equipos multidisciplinarios. - Enfoque en cumplimiento de KPIs y desarrollo de cultura basada en datos. ## Experiencia Laboral - **Director de Transformacion Digital en TechCorp**: implementacion de estrategias de innovacion. - **Gerente de Proyectos en Empresa XYZ**: liderazgo de equipos en telecomunicaciones y finanzas. - **Consultor Senior en Deloitte**: asesoria en analitica de datos y gestion de riesgos. ## Logros Destacados - Reduccion de costos operativos en un 20% mediante procesos de digitalizacion. - Implementacion de tableros de control para seguimiento de indicadores clave. - Desar

#Gestión de índices de Pinecone

Convierte nombres de archivo en nombres válidos para índices Pinecone.

Lista los índices existentes.

Crea un índice si no existe.

In [59]:
# Helpers de índices por CV

def sanitize_index_name(name: str) -> str:
    """
    Pinecone index name: lowercase, alfanumérico y guiones, 1-45 chars, comenzar con letra.
    Creamos: f"cv_{slug}" donde slug se arma desde el nombre del archivo (sin extensión).
    """
    slug = name.lower()
    slug = re.sub(r"\s+", "-", slug)
    slug = re.sub(r"[^a-z0-9-]", "-", slug)
    slug = re.sub(r"-{2,}", "-", slug).strip("-")
    if not slug or not slug[0].isalpha():
        slug = "a" + slug
    slug = slug[:40]  # dejamos margen para "cv_"
    return f"{INDEX_PREFIX}{slug}"

def list_cv_indices() -> List[str]:
    listing = pc.list_indexes()
    names = [item["name"] for item in listing] if isinstance(listing, list) else [d.get("name") for d in listing]
    return sorted([n for n in names if n and n.startswith(INDEX_PREFIX)])

def ensure_index(index_name: str):
    existing = list_cv_indices()
    if index_name not in existing:
        pc.create_index(
            name=index_name,
            dimension=EMB_DIM,
            metric="cosine",
            spec=ServerlessSpec(
                cloud=PINECONE_CLOUD,
                region=PINECONE_REGION
            ),
            vector_type=VectorType.DENSE
        )


#Carga de documentos a Pinecone

Para cada .txt o .pdf:

Limpia y divide el texto.

Calcula embeddings.

Crea vectores con metadatos.

Hace upsert (insertar o actualizar) en el índice correspondiente (uno por archivo).

In [60]:
# Ingesta/Upsert: UN índice por CV

def upsert_documents_per_cv(folder: str) -> str:
    os.makedirs(folder, exist_ok=True)
    paths = [Path(p) for p in glob.glob(os.path.join(folder, "*.pdf"))] + \
            [Path(p) for p in glob.glob(os.path.join(folder, "*.txt"))]
    if not paths:
        return "No se encontraron documentos (.pdf/.txt) en la carpeta."

    total_chunks = 0
    created = 0
    updated = 0

    for path in paths:
        base_noext = path.stem
        index_name = sanitize_index_name(base_noext)

        # Asegurar índice para este CV
        before = set(list_cv_indices())
        ensure_index(index_name)
        after = set(list_cv_indices())
        if index_name in (after - before):
            created += 1
        else:
            updated += 1

        # Leer y trocear
        raw_text = read_file(path).strip()
        if not raw_text:
            print(f"Vacío o ilegible: {path.name}")
            continue

        chunks = chunk_text(raw_text)
        if not chunks:
            print(f"Sin chunks: {path.name}")
            continue

        vecs = embed_texts(chunks)
        vectors = []
        for i, vec in enumerate(vecs):
            vid = f"{base_noext}-{i:04d}"
            vectors.append({
                "id": vid,
                "values": vec,
                "metadata": {
                    "texto": chunks[i],
                    "archivo": path.name,
                    "chunk_id": i,
                    "fecha": datetime.today().strftime("%Y-%m-%d")
                }
            })

        # Upsert al índice del CV
        idx = pc.Index(index_name)
        # Upsert por lotes
        BATCH = 200
        for i in range(0, len(vectors), BATCH):
            idx.upsert(vectors=vectors[i:i+BATCH])

        total_chunks += len(chunks)

    return f"Indexado OK. Archivos: {len(paths)} | Índices nuevos: {created} | Actualizados: {updated} | Chunks insertados: {total_chunks}"

#Consulta por vector (Retrieve)

Usa el vector de la query para buscar en un índice de Pinecone.

Devuelve los top_k matches.

Arma texto con los resultados y sus fuentes.

In [61]:
# Retrieve (consulta vectorial) desde un índice concreto

def retrieve_from_index(index_name: str, query: str, top_k: int = 4) -> List[Dict[str, Any]]:
    idx = pc.Index(index_name)
    qvec = embed_text(query)
    res = idx.query(vector=qvec, top_k=int(top_k), include_metadata=True)

    matches = res.get("matches", []) if isinstance(res, dict) else getattr(res, "matches", [])
    out = []
    for m in matches or []:
        score = m.get("score") if isinstance(m, dict) else getattr(m, "score", None)
        meta = m.get("metadata", {}) if isinstance(m, dict) else getattr(m, "metadata", {}) or {}
        out.append({
            "score": round(float(score or 0.0), 4),
            "texto": meta.get("texto", ""),
            "archivo": meta.get("archivo", "")
        })
    return out

def build_context(matches: List[Dict[str, Any]]) -> str:
    if not matches:
        return "(no se recuperaron pasajes relevantes)"
    parts = []
    for i, m in enumerate(matches, 1):
        parts.append(f"[{i}] ({m['archivo']}, score={m['score']})\n{m['texto']}")
    return "\n\n---\n\n".join(parts)

#Router multiagente

Usa un LLM (Groq) para decidir cuál índice (archivo) es más relevante para una pregunta.

Solo devuelve el nombre del índice (o NONE).

In [62]:
# Router Agent: elegir ÍNDICE del CV

def router_agent(user_question: str, candidates: List[str]) -> Optional[str]:
    if not candidates:
        return None
    cand_list = "\n".join(f"- {c}" for c in candidates)
    messages = [
        {"role": "system", "content": ROUTER_SYSTEM},
        {"role": "user", "content": f"Índices disponibles:\n{cand_list}\n\nPregunta: {user_question}\n\nDevuelve SOLO un nombre EXACTO de la lista, o 'NONE' si ninguno aplica."}
    ]
    try:
        r = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=messages,
            temperature=0.0,
            max_tokens=50
        )
        choice = (r.choices[0].message.content or "").strip().splitlines()[0].strip()
        if choice.upper() == "NONE":
            return None
        return choice if choice in candidates else None
    except Exception:
        return None

#Verificar chunks

Muestra el primer chunk de cada archivo.

Sirve como herramienta de debugging.

In [63]:
# Verificación de chunks (primer chunk por CV)
def verificar_chunks():
    import os
    from langchain.text_splitter import RecursiveCharacterTextSplitter

    textos_por_cv = []
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

    for fname in os.listdir(DOCS_DIR):
        filepath = os.path.join(DOCS_DIR, fname)
        if not os.path.isfile(filepath):
            continue
        if not (fname.endswith(".txt") or fname.endswith(".pdf")):
            continue

        # Leer archivo con robustez
        try:
            with open(filepath, "r", encoding="utf-8") as f:
                text = f.read()
        except UnicodeDecodeError:
            with open(filepath, "r", encoding="latin-1", errors="ignore") as f:
                text = f.read()

        # Limpiar texto
        text = text.replace("\x00", "").strip()

        # Dividir en chunks
        chunks = splitter.split_text(text)
        primer_chunk = chunks[0] if chunks else "(vacío o ilegible)"
        textos_por_cv.append(f"**{fname}**\n{primer_chunk}")

    return "\n\n---\n\n".join(textos_por_cv) or " No se encontraron archivos válidos."

In [64]:
verificar_chunks()

'**cv1.txt**\n# CV Corporativo - Ricardo Lopez\n\n## Datos Personales\n- Nombre: Ricardo Lopez  \n- Edad: 47 anios  \n- Correo: ricardo.lopez.corporativo@example.com  \n- Ubicacion: Buenos Aires, Argentina  \n- LinkedIn: linkedin.com/in/ricardolopez-corporativo\n\n---\n\n**cv2.txt**\n# CV Tecnologico - Sofia Martinez\n\n## Datos Personales\n- Nombre: Sofia Martinez  \n- Edad: 38 anios  \n- Correo: sofia.martinez.tecnologico@example.com  \n- Ubicacion: Mendoza, Argentina  \n- LinkedIn: linkedin.com/in/sofiamartinez-tecnologico  \n\n## Resumen Profesional\n- Especialista en desarrollo de software, arquitecturas de datos e inteligencia artificial.  \n- Experiencia en startups y empresas multinacionales.  \n- Enfoque en soluciones escalables, seguras y eficientes en la nube.'

#Chat multi-agente (RAG)

Usa el router para elegir índice.

Recupera contexto con vectores.

Llama al modelo LLM con contexto, pregunta e historial.

Devuelve respuesta + fuentes.

In [65]:
list_cv_indices()

[]

In [46]:
def chat(user_message, chat_history):
    """
    - Router elige el índice (CV) más adecuado.
    - Retrieve desde ese índice (usando PARAMS['k']).
    - Groq responde con el contexto recuperado (usando temp/top_p/max_tokens de PARAMS).
    - Devuelve UN string (respuesta + fuentes).
    """
    # Lee parámetros actuales elegidos por el usuario
    K = int(PARAMS["k"])
    TEMPERATURE = float(PARAMS["temperature"])
    TOP_P = float(PARAMS["top_p"])
    MAX_TOKENS = int(PARAMS["max_tokens"])

    # 1) Router: elegir índice de CV
    candidates = list_cv_indices()
    chosen_index = router_agent(user_message, candidates)

    # 2) Retrieve desde el índice elegido (o mensaje si no hay)
    if chosen_index:
        matches = retrieve_from_index(chosen_index, user_message, top_k=K)
        chosen_info = chosen_index
    else:
        matches = []
        chosen_info = "— (sin selección)"

    context_text = build_context(matches)

    # 3) Armar mensajes para Groq
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    messages.append({"role": "system", "content": f"Índice seleccionado por el router: {chosen_info}"})
    messages.append({"role": "system", "content": f"Contexto recuperado:\n\n{context_text}"})

    for u, b in history:
        messages.append({"role": "user", "content": u})
        messages.append({"role": "assistant", "content": b})
    messages.append({"role": "user", "content": user_message})

    # 4) Llamada Groq con parámetros elegidos
    resp = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=messages,
        temperature=TEMPERATURE,
        top_p=TOP_P,
        max_tokens=MAX_TOKENS
    )
    bot_reply = resp.choices[0].message.content

    # 5) Añadir pie de fuentes en el MISMO string
    if matches:
        fuentes = "\n".join([f"• {m['archivo']} (score={m['score']})" for m in matches])
    else:
        fuentes = "—"
    bot_reply_with_sources = (
        f"{bot_reply}\n\n---\n"
        f"**Router → índice seleccionado:** {chosen_info}\n"
        f"**Fuentes (pasajes recuperados):**\n{fuentes}"
    )

    # 6) Guardar historial (nuestra simulación de contexto)
    history.append((user_message, bot_reply_with_sources))

    return bot_reply_with_sources

#Memoria del Chat

Y se actualiza en la última línea de la función chat():

La memoria se acumula en esta variable global:

Cada vez que el usuario envía un mensaje, se guarda el par (pregunta, respuesta) en history.

Esta history se reutiliza en cada nuevo mensaje para simular “memoria del chat”.

La historia se convierte en parte del prompt enviado al modelo LLM (Groq):

for u, b in history:

    messages.append({"role": "user", "content": u})
    messages.append({"role": "assistant", "content": b})

Esto permite que el modelo entienda el contexto conversacional anterior.

Memoria del Chat

El modelo que estás usando es: "llama3-8b-8192"

Esto significa que su ventana de contexto es de 8192 tokens (máximo que puede procesar en cada mensaje).

Todo lo siguiente consume tokens:

System prompts (prompts iniciales)

Texto recuperado por RAG (context_text)

Historial (history)

Nueva pregunta del usuario

Si todo eso suma más de 8192 tokens, el modelo recortará automáticamente el prompt, y puede ignorar partes viejas del historial o contexto.

In [28]:
history = []  # [(user, bot), ...]

In [30]:
from typing import List, Dict, Any, Optional, Tuple


In [31]:
# =========================
# Versión extendida: Multiagente y agente único
# =========================

def router_agent_single(user_question: str, candidates: List[str]) -> Optional[str]:
    """
    Elige UN único índice de CV según la pregunta del usuario.
    """
    if not candidates:
        return None
    cand_list = "\n".join(f"- {c}" for c in candidates)
    messages = [
        {"role": "system", "content": ROUTER_SYSTEM},
        {"role": "user", "content": f"\u00cdndices disponibles:\n{cand_list}\n\nPregunta: {user_question}\n\nDevuelve SOLO un nombre EXACTO de la lista, o 'NONE' si ninguno aplica."}
    ]
    try:
        r = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=messages,
            temperature=0.0,
            max_tokens=50
        )
        choice = (r.choices[0].message.content or "").strip().splitlines()[0].strip()
        return choice if choice.upper() != "NONE" and choice in candidates else None
    except Exception:
        return None

def router_agent_multi(user_question: str, candidates: List[str]) -> List[str]:
    """
    Devuelve una lista de índices relevantes según la pregunta del usuario.
    """
    if not candidates:
        return []
    cand_list = "\n".join(f"- {c}" for c in candidates)
    system_prompt = (
        "Eres un enrutador inteligente. Tu tarea es elegir todos los índices de la lista que sean relevantes para la consulta del usuario.\n"
        "Responde con una lista separada por comas, usando exactamente los nombres tal como aparecen.\n"
        "Si ninguno aplica, responde 'NONE'."
    )
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"\u00cdndices disponibles:\n{cand_list}\n\nPregunta: {user_question}\n\nDevuelve una lista separada por comas o 'NONE'."}
    ]
    try:
        r = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=messages,
            temperature=0.0,
            max_tokens=100
        )
        raw = (r.choices[0].message.content or "").strip()
        if raw.upper() == "NONE":
            return []
        selected = [s.strip() for s in raw.split(",") if s.strip() in candidates]
        return selected
    except Exception:
        return []

def chat_multiagent(user_message: str, chat_history: List[Tuple[str, str]]) -> str:
    """
    - Usa el router multiagente para detectar todos los CVs relevantes.
    - Consulta a cada índice por separado y concatena el contexto.
    - Genera una única respuesta compuesta con fuentes diferenciadas.
    """
    K = int(PARAMS["k"])
    TEMPERATURE = float(PARAMS["temperature"])
    TOP_P = float(PARAMS["top_p"])
    MAX_TOKENS = int(PARAMS["max_tokens"])

    candidates = list_cv_indices()
    selected_indices = router_agent_multi(user_message, candidates)

    all_matches = []
    all_contexts = []
    index_labels = []

    for idx in selected_indices:
        matches = retrieve_from_index(idx, user_message, top_k=K)
        context = build_context(matches)
        all_matches.extend(matches)
        index_labels.append(idx)
        all_contexts.append(f"## Contexto de {idx}\n\n{context}")

    combined_context = "\n\n".join(all_contexts) if all_contexts else "(no se recuperó contexto)"

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "system", "content": f"Índices seleccionados: {', '.join(index_labels) if index_labels else '— (ninguno)'}"},
        {"role": "system", "content": f"Contexto recuperado:\n\n{combined_context}"}
    ]

    for u, b in chat_history:
        messages.append({"role": "user", "content": u})
        messages.append({"role": "assistant", "content": b})
    messages.append({"role": "user", "content": user_message})

    resp = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=messages,
        temperature=TEMPERATURE,
        top_p=TOP_P,
        max_tokens=MAX_TOKENS
    )
    bot_reply = resp.choices[0].message.content

    if all_matches:
        fuentes = "\n".join([f"• {m['archivo']} (score={m['score']})" for m in all_matches])
    else:
        fuentes = "—"

    bot_reply_with_sources = (
        f"{bot_reply}\n\n---\n"
        f"**Router → índices seleccionados:** {', '.join(index_labels) if index_labels else '—'}\n"
        f"**Fuentes:**\n{fuentes}"
    )

    chat_history.append((user_message, bot_reply_with_sources))
    return bot_reply_with_sources


In [36]:
PARAMS = {
    "k": 10,
    "temperature": 0.7,
    "top_p": 0.7,
    "max_tokens": 512
}

In [67]:
folder = "/content/cv"  # o donde tengas tus .pdf/.txt
msg = upsert_documents_per_cv(folder)
print(msg)

Indexado OK. Archivos: 2 | Índices nuevos: 2 | Actualizados: 0 | Chunks insertados: 16


In [68]:
list_cv_indices()

['cv--cv1', 'cv--cv2']

In [69]:
chat_multiagent("resume el cv de sofia", [])


'Contexto de cv--cv2\n\nSofia Martinez:\n\n* Especialista en desarrollo de software, arquitecturas de datos e inteligencia artificial\n* Experiencia en sistemas distribuidos, automatización de procesos empresariales mediante RPA y despliegue de modelos de IA con MLOps en entornos cloud (AWS, GCP)\n* Diplomatura en Deep Learning - MIT\n* Experiencia en startups y empresas multinacionales\n* Enfoque en soluciones escalables, seguras y eficientes en la nube\n\n---\n**Router → índices seleccionados:** cv--cv1, cv--cv2\n**Fuentes:**\n• cv1.txt (score=0.3898)\n• cv1.txt (score=0.3887)\n• cv1.txt (score=0.2385)\n• cv1.txt (score=0.2352)\n• cv2.txt (score=0.5111)\n• cv2.txt (score=0.4835)\n• cv2.txt (score=0.2532)\n• cv2.txt (score=0.2049)'

In [70]:
chat_multiagent("resume el cv de ricardo", [])

'Según el contexto, el CV de Ricardo Lopez es el siguiente:\n\n* Es un ejecutivo con experiencia en liderazgo empresarial y gestión de proyectos tecnológicos.\n* Especialista en transformación digital y gestión de equipos multidisciplinarios.\n* Enfoque en cumplimiento de KPIs y desarrollo de cultura basada en datos.\n* Experiencia laboral como Director de Transformación Digital en TechCorp y Gerente de Proyectos en Empresa XYZ.\n* Implementó estrategias de innovación y lideró equipos en telecomunicaciones y otros sectores.\n* Ubicado en Buenos Aires, Argentina y tiene un perfil en LinkedIn.\n\n---\n**Router → índices seleccionados:** cv--cv1, cv--cv2\n**Fuentes:**\n• cv1.txt (score=0.5742)\n• cv1.txt (score=0.4811)\n• cv1.txt (score=0.2956)\n• cv1.txt (score=0.2463)\n• cv2.txt (score=0.4253)\n• cv2.txt (score=0.3512)\n• cv2.txt (score=0.2441)\n• cv2.txt (score=0.173)'

In [72]:
chat_multiagent("Cual de los dos candidatos es mas grande en edad", [])

'Según el contexto, Ricardo Lopez (cv--cv1) tiene 47 años, mientras que Sofia Martinez (cv--cv2) tiene 38 años. Por lo tanto, Ricardo Lopez es el candidato más grande en edad.\n\n---\n**Router → índices seleccionados:** cv--cv1, cv--cv2\n**Fuentes:**\n• cv1.txt (score=0.3716)\n• cv1.txt (score=0.3489)\n• cv1.txt (score=0.3201)\n• cv1.txt (score=0.3084)\n• cv2.txt (score=0.3146)\n• cv2.txt (score=0.2824)\n• cv2.txt (score=0.2813)\n• cv2.txt (score=0.2049)'

#Interfaz de usuario (Gradio)

Incluye:

Botón para reindexar.

Sliders para k, temp, top_p, max_tokens.

Chat interface.

Herramienta para verificar chunks.

In [79]:
# =========================
# Interfaz Gradio final — Multiagente con ejemplos y limpieza
# =========================

PARAMS = {
    "k": TOP_K_DEFAULT,
    "temperature": TEMPERATURE_DEFAULT,
    "top_p": TOP_P_DEFAULT,
    "max_tokens": MAX_TOKENS_DEFAULT,
}

chat_history = []

# Reindexar documentos

def reindex_action():
    os.makedirs(DOCS_DIR, exist_ok=True)
    return upsert_documents_per_cv(DOCS_DIR)

# Borrar todos los índices (para limpieza completa)
def reset_all():
    deleted = []
    for idx in list_cv_indices():
        try:
            pc.delete_index(idx)
            deleted.append(idx)
        except Exception:
            pass
    chat_history.clear()
    return f"Se eliminaron los índices: {', '.join(deleted)}\nHistorial reiniciado."

# Borrar solo el historial

def limpiar_chat():
    chat_history.clear()
    return [], ""

# Callbacks sliders

def _set_k(v):           PARAMS["k"] = int(v)
def _set_temp(v):        PARAMS["temperature"] = float(v)
def _set_top_p(v):       PARAMS["top_p"] = float(v)
def _set_max_tokens(v):  PARAMS["max_tokens"] = int(v)

# Interfaz
with gr.Blocks() as demo:
    gr.Markdown("## RAG multi-agente por CV — Groq + Pinecone")
    gr.Markdown("Subí tus CVs en `/content/cv` (.pdf o .txt) y presioná **Reindexar** para crear un índice por cada CV.")

    with gr.Row():
        re_btn = gr.Button("Reindexar (/content/cv)")
        reset_btn = gr.Button("🧹 Limpiar todo (historial + índices)")
    re_out = gr.Textbox(label="Estado", lines=3)
    re_btn.click(fn=reindex_action, outputs=[re_out])
    reset_btn.click(fn=reset_all, outputs=[re_out])

    gr.Markdown("---")
    gr.Markdown("### Parámetros de recuperación y generación")

    with gr.Row():
        k_slider = gr.Slider(label="Top-K retrieve", minimum=1, maximum=10, step=1, value=PARAMS["k"])
        temp_slider = gr.Slider(label="Temperature", minimum=0.0, maximum=1.5, step=0.1, value=PARAMS["temperature"])
        top_p_slider = gr.Slider(label="Top-p", minimum=0.1, maximum=1.0, step=0.05, value=PARAMS["top_p"])
        max_tk_slider = gr.Slider(label="Max tokens", minimum=64, maximum=2048, step=32, value=PARAMS["max_tokens"])

    k_slider.change(_set_k, inputs=[k_slider])
    temp_slider.change(_set_temp, inputs=[temp_slider])
    top_p_slider.change(_set_top_p, inputs=[top_p_slider])
    max_tk_slider.change(_set_max_tokens, inputs=[max_tk_slider])

    gr.Markdown("---")
    gr.Markdown("### Modo de operación")
    modo_multi = gr.Checkbox(label="Usar modo multiagente (varios CVs) 💡", value=True)

    gr.Markdown("### Chat con el sistema")
    with gr.Row():
        msg = gr.Textbox(label="Tu pregunta", lines=2, scale=8)
        send_btn = gr.Button("Enviar", scale=2)

    chatbox = gr.Chatbot()
    clear_btn = gr.Button("🧼 Limpiar chat")

    # Ejemplos
    ejemplos = [
        "¿En qué proyectos de ciencia de datos participó Ricardo?",
        "¿Qué experiencia tienen Sofía y Ricardo con sklearn, XGBoost o LightGBM?",
        "¿Cuál de los candidatos tiene un perfil más académico?",
        "¿Cuál de los candidatos tiene un perfil más corporativo?",
        "¿Quién de los candidatos sabe Python y trabajó en minería?",
        "¿Qué personas tienen experiencia en proyectos internacionales y manejo de AWS o GCP?",
        "Estoy buscando alguien con conocimientos en ML, datos y experiencia minera. ¿A quién considero?",
        "¿Quiénes lideraron equipos técnicos y usaron Power BI o Tableau?"
    ]
    gr.Examples(examples=ejemplos, inputs=msg)

    # Lógica de respuesta
    def responder(input_text, modo_multi):
          if not input_text.strip():
              return []

          print("🔧 Parámetros actuales:")
          print(PARAMS)

          if modo_multi:
              respuesta = chat_multiagent(input_text, chat_history)
          else:
              respuesta = chat(input_text, chat_history)

          return [(input_text, respuesta)]

    send_btn.click(fn=responder, inputs=[msg, modo_multi], outputs=[chatbox])
    msg.submit(fn=responder, inputs=[msg, modo_multi], outputs=[chatbox]).then(
        fn=lambda: "", inputs=None, outputs=msg
    )

    clear_btn.click(fn=limpiar_chat, outputs=[chatbox, msg])

    gr.Markdown("---")
    gr.Markdown("### Vista de chunks iniciales")
    verif_btn = gr.Button("Verificar primer chunk de cada CV")
    verif_out = gr.Textbox(label="Preview de chunks", lines=10)
    verif_btn.click(fn=verificar_chunks, outputs=[verif_out])

demo.launch()


/tmp/ipython-input-150938962.py:80: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbox = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://54eb75b60747bd00d9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#DEBUGGING VERIFICACION MANUAL

Es para ejecutar paso a paso todo el flujo con un archivo específico, esto para analizar los errores que fueron surgiendo:

Lee archivo.

Divide en chunks.

Calcula embeddings.

Hace upsert en Pinecone.

Elimina índices si es necesario.

In [273]:
from pathlib import Path
from pprint import pprint

archivo = "/content/cv/cv1.txt"
text = read_txt(archivo)
print("Texto leído:")
pprint(text[:1000])

chunks = chunk_text(text)
print(f"\nChunks generados: {len(chunks)}")
for i, c in enumerate(chunks[:3]):
    print(f"--- Chunk {i+1} ---\n{c}\n")


Texto leído:
('# CV Corporativo - Ricardo Lopez ## Datos Personales - Nombre: Ricardo Lopez '
 '- Edad: 47 anios - Correo: ricardo.lopez.corporativo@example.com - '
 'Ubicacion: Buenos Aires, Argentina - LinkedIn: '
 'linkedin.com/in/ricardolopez-corporativo ## Resumen Profesional - Ejecutivo '
 'con experiencia en liderazgo empresarial y gestion de proyectos '
 'tecnologicos. - Especialista en transformacion digital y gestion de equipos '
 'multidisciplinarios. - Enfoque en cumplimiento de KPIs y desarrollo de '
 'cultura basada en datos. ## Experiencia Laboral - **Director de '
 'Transformacion Digital en TechCorp**: implementacion de estrategias de '
 'innovacion. - **Gerente de Proyectos en Empresa XYZ**: liderazgo de equipos '
 'en telecomunicaciones y finanzas. - **Consultor Senior en Deloitte**: '
 'asesoria en analitica de datos y gestion de riesgos. ## Logros Destacados - '
 'Reduccion de costos operativos en un 20% mediante procesos de '
 'digitalizacion. - Implementacion de 

In [274]:
!file -i /content/cv/!file -i /content/cv/*.txt

/content/cv/!file:   cannot open `/content/cv/!file' (No such file or directory)
/content/cv/cv1.txt: text/plain; charset=us-ascii
/content/cv/cv2.txt: text/plain; charset=us-ascii


In [275]:
with open("/content/cv/cv1.txt", encoding="latin-1") as f:
    print(f.read()[:500])


# CV Corporativo - Ricardo Lopez

## Datos Personales
- Nombre: Ricardo Lopez  
- Edad: 47 anios  
- Correo: ricardo.lopez.corporativo@example.com  
- Ubicacion: Buenos Aires, Argentina  
- LinkedIn: linkedin.com/in/ricardolopez-corporativo  

## Resumen Profesional
- Ejecutivo con experiencia en liderazgo empresarial y gestion de proyectos tecnologicos.  
- Especialista en transformacion digital y gestion de equipos multidisciplinarios.  
- Enfoque en cumplimiento de KPIs y desarrollo de cultur


In [276]:
try:
    vecs = embed_texts(chunks)
    print(f"{len(vecs)} vectores de embeddings generados.")
    print(vecs[0][:5])  # Mostrar los primeros 5 valores del primer vector
except Exception as e:
    print(f"Error al generar embeddings: {e}")



8 vectores de embeddings generados.
[-0.02830718830227852, -0.03697291761636734, -0.043179091066122055, 0.014025023207068443, 0.09206068515777588]


In [277]:
try:
    BATCH = 200
    for i in range(0, len(vectors), BATCH):
        print(f"Upserting batch {i}–{i+BATCH} en índice '{index_name}'")
        idx.upsert(vectors=vectors[i:i+BATCH])
except Exception as e:
    print(f"Error al hacer upsert en Pinecone: {e}")


Upserting batch 0–200 en índice 'cv--cv1'
Error al hacer upsert en Pinecone: 'NoneType' object is not callable


In [278]:
indices = pc.list_indexes()
for idx in indices:
    nombre = idx["name"]
    if nombre.startswith("cv-"):
        pc.delete_index(nombre)

In [279]:
from pathlib import Path

path = Path("/content/cv/cv1.txt")

base_noext = path.stem
index_name = sanitize_index_name(base_noext)

print(f"\n Procesando archivo: {path.name}")
print(f"Índice: {index_name}")

ensure_index(index_name)

raw_text = read_file(path).strip()
if not raw_text:
    print(f"Vacío o ilegible: {path.name}")
else:
    chunks = chunk_text(raw_text)
    if not chunks:
        print(f"Sin chunks: {path.name}")
    else:
        print(f"{len(chunks)} chunks generados")
        try:
            vecs = embed_texts(chunks)
            print(f"{len(vecs)} vectores generados")
            vectors = []
            for i, vec in enumerate(vecs):
                vid = f"{base_noext}-{i:04d}"
                vectors.append({
                    "id": vid,
                    "values": vec,
                    "metadata": {
                        "texto": chunks[i],
                        "archivo": path.name,
                        "chunk_id": i,
                        "fecha": datetime.today().strftime("%Y-%m-%d")
                    }
                })

            idx = pc.Index(index_name)
            for i in range(0, len(vectors), 200):
                print(f"Subiendo batch {i}–{i+200}")
                idx.upsert(vectors=vectors[i:i+200])
        except Exception as e:
            print(f"Error al generar embeddings o subir: {e}")



 Procesando archivo: cv1.txt
Índice: cv--cv1
8 chunks generados
8 vectores generados
Subiendo batch 0–200
